# Imports 

In [1]:
!pip install tables==3.6.1
!pip install socceraction==0.2.0

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib.patches import Ellipse
import seaborn as sns
from math import *
import matplotlib.pylab as pyl
import pickle
import swifter
import warnings
import plotly.express as px
from itertools import chain
import scipy.stats as sps
from tqdm import tqdm
from unidecode import unidecode
import re
from io import BytesIO
from pathlib import Path
from tqdm.notebook import tqdm
from urllib.parse import urlparse
from urllib.request import urlopen, urlretrieve
from zipfile import ZipFile, is_zipfile
import pandas as pd
from sklearn.metrics import brier_score_loss, roc_auc_score  # version 0.22.2
from xgboost import XGBClassifier  # version 1.0.2

import socceraction.vaep.features as features
import socceraction.vaep.labels as labels

from socceraction.spadl.wyscout import convert_to_spadl
from socceraction.vaep.formula import value

In [3]:
#pd.set_option('max_colwidth', 999)
pd.set_option('display.max_columns', 1000)
pd.set_option("display.max_rows", 3000)

In [4]:
import warnings
warnings.filterwarnings('ignore', category=pd.io.pytables.PerformanceWarning)

# Preprocess the data

## Preprocess the Wyscout data

In [5]:
def read_json_file(filename):
    with open(filename, 'rb') as json_file:
        return BytesIO(json_file.read()).getvalue().decode('unicode_escape')

### Teams

In [6]:
json_teams = read_json_file('teams.json')
df_teams = pd.read_json(json_teams)

In [7]:
df_teams.head(10)

,city,name,wyId,officialName,area,type
0,Newcastle upon Tyne,Newcastle United,1613,Newcastle United FC,"{'name': 'England', 'id': '0', 'alpha3code': '...",club
1,Vigo,Celta de Vigo,692,Real Club Celta de Vigo,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
2,Barcelona,Espanyol,691,Reial Club Deportiu Espanyol,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
3,Vitoria-Gasteiz,Deportivo Alavés,696,Deportivo Alavés,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
4,Valencia,Levante,695,Levante UD,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
5,Troyes,Troyes,3795,Espérance Sportive Troyes Aube Champagne,"{'name': 'France', 'id': '250', 'alpha3code': ...",club
6,Getafe (Madrid),Getafe,698,Getafe Club de Fútbol,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
7,Mönchengladbach,Borussia M'gladbach,2454,Borussia VfL Mönchengladbach,"{'name': 'Germany', 'id': '276', 'alpha3code':...",club
8,"Huddersfield, West Yorkshire",Huddersfield Town,1673,Huddersfield Town FC,"{'name': 'England', 'id': '0', 'alpha3code': '...",club
9,Bilbao,Athletic Club,678,Athletic Club Bilbao,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club


In [8]:
df_teams.to_hdf('wyscout.h5', key='teams', mode='w')

### Players

In [9]:
json_players = read_json_file('players.json')
df_players = pd.read_json(json_players)

In [10]:
df_players.head(10)

,passportArea,weight,firstName,middleName,lastName,currentTeamId,birthDate,height,role,birthArea,wyId,foot,shortName,currentNationalTeamId
0,"{'name': 'Turkey', 'id': '792', 'alpha3code': ...",78,Harun,,Tekin,4502,1989-06-17,187,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'Turkey', 'id': '792', 'alpha3code': ...",32777,right,H. Tekin,4687
1,"{'name': 'Senegal', 'id': '686', 'alpha3code':...",73,Malang,,Sarr,3775,1999-01-23,182,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'France', 'id': '250', 'alpha3code': ...",393228,left,M. Sarr,4423
2,"{'name': 'France', 'id': '250', 'alpha3code': ...",72,Over,,Mandanda,3772,1998-10-26,176,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'France', 'id': '250', 'alpha3code': ...",393230,,O. Mandanda,null
3,"{'name': 'Senegal', 'id': '686', 'alpha3code':...",82,Alfred John Momar,,N'Diaye,683,1990-03-06,187,"{'code2': 'MD', 'code3': 'MID', 'name': 'Midfi...","{'name': 'France', 'id': '250', 'alpha3code': ...",32793,right,A. N'Diaye,19314
4,"{'name': 'France', 'id': '250', 'alpha3code': ...",84,Ibrahima,,Konaté,2975,1999-05-25,192,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'France', 'id': '250', 'alpha3code': ...",393247,right,I. Konaté,null
5,"{'name': 'Netherlands', 'id': '528', 'alpha3co...",83,Jasper,,Cillessen,676,1989-04-22,185,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'Netherlands', 'id': '528', 'alpha3co...",33,right,J. Cillessen,664
6,"{'name': 'Belgium', 'id': '56', 'alpha3code': ...",91,Toby,,Alderweireld,1624,1989-03-02,187,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'Belgium', 'id': '56', 'alpha3code': ...",36,right,T. Alderweireld,5629
7,"{'name': 'Belgium', 'id': '56', 'alpha3code': ...",88,Jan,,Vertonghen,1624,1987-04-24,189,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'Belgium', 'id': '56', 'alpha3code': ...",48,left,J. Vertonghen,5629
8,"{'name': 'France', 'id': '250', 'alpha3code': ...",74,Alexander,,Djiku,3783,1994-08-09,182,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'France', 'id': '250', 'alpha3code': ...",229427,right,A. Djiku,null
9,"{'name': 'Denmark', 'id': '208', 'alpha3code':...",76,Christian,,Dannemann Eriksen,1624,1992-02-14,180,"{'code2': 'MD', 'code3': 'MID', 'name': 'Midfi...","{'name': 'Denmark', 'id': '208', 'alpha3code':...",54,right,C. Eriksen,7712


In [11]:
df_players.to_hdf('wyscout.h5', key='players', mode='a')

### Matches

In [12]:
competitions = [
     'England',
#     'France',
#     'Germany',
#     'Italy',
#     'Spain',
#    'European Championship',
#     'World Cup'
]

In [13]:
dfs_matches = []
for competition in competitions:
    competition_name = competition.replace(' ', '_')
    file_matches = f'matches_{competition_name}.json'
    json_matches = read_json_file(file_matches)
    df_matches = pd.read_json(json_matches)
    dfs_matches.append(df_matches)
df_matches = pd.concat(dfs_matches)

In [14]:
df_matches.head(10)

,status,roundId,gameweek,teamsData,seasonId,dateutc,winner,venue,wyId,label,date,referees,duration,competitionId
0,Played,4405654,38,"{'1646': {'scoreET': 0, 'coachId': 8880, 'side...",181150,2018-05-13 14:00:00,1659,Turf Moor,2500089,"Burnley - AFC Bournemouth, 1 - 2","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 385705, 'role': 'referee'}, {'r...",Regular,364
1,Played,4405654,38,"{'1628': {'scoreET': 0, 'coachId': 8357, 'side...",181150,2018-05-13 14:00:00,1628,Selhurst Park,2500090,"Crystal Palace - West Bromwich Albion, 2 - 0","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 381851, 'role': 'referee'}, {'r...",Regular,364
2,Played,4405654,38,"{'1609': {'scoreET': 0, 'coachId': 7845, 'side...",181150,2018-05-13 14:00:00,1609,The John Smith's Stadium,2500091,"Huddersfield Town - Arsenal, 0 - 1","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 384965, 'role': 'referee'}, {'r...",Regular,364
3,Played,4405654,38,"{'1651': {'scoreET': 0, 'coachId': 8093, 'side...",181150,2018-05-13 14:00:00,1612,Anfield,2500092,"Liverpool - Brighton & Hove Albion, 4 - 0","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 385704, 'role': 'referee'}, {'r...",Regular,364
4,Played,4405654,38,"{'1644': {'scoreET': 0, 'coachId': 93112, 'sid...",181150,2018-05-13 14:00:00,1611,Old Trafford,2500093,"Manchester United - Watford, 1 - 0","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 381853, 'role': 'referee'}, {'r...",Regular,364
5,Played,4405654,38,"{'1613': {'scoreET': 0, 'coachId': 210700, 'si...",181150,2018-05-13 14:00:00,1613,St. James' Park,2500094,"Newcastle United - Chelsea, 3 - 0","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 384888, 'role': 'referee'}, {'r...",Regular,364
6,Played,4405654,38,"{'1625': {'scoreET': 0, 'coachId': 267136, 'si...",181150,2018-05-13 14:00:00,1625,St. Mary's Stadium,2500095,"Southampton - Manchester City, 0 - 1","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 385911, 'role': 'referee'}, {'r...",Regular,364
7,Played,4405654,38,"{'10531': {'scoreET': 0, 'coachId': 32573, 'si...",181150,2018-05-13 14:00:00,1639,Liberty Stadium,2500096,"Swansea City - Stoke City, 1 - 2","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 378952, 'role': 'referee'}, {'r...",Regular,364
8,Played,4405654,38,"{'1631': {'scoreET': 0, 'coachId': 209010, 'si...",181150,2018-05-13 14:00:00,1624,Wembley Stadium,2500097,"Tottenham Hotspur - Leicester City, 5 - 4","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 378951, 'role': 'referee'}, {'r...",Regular,364
9,Played,4405654,38,"{'1623': {'scoreET': 0, 'coachId': 8541, 'side...",181150,2018-05-13 14:00:00,1633,London Stadium,2500098,"West Ham United - Everton, 3 - 1","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 408156, 'role': 'referee'}, {'r...",Regular,364


In [15]:
df_matches.to_hdf('wyscout.h5', key='matches', mode='a')

### Events

In [16]:
for competition in competitions:
    competition_name = competition.replace(' ', '_')
    file_events = f'events_{competition_name}.json'
    json_events = read_json_file(file_events)
    df_events = pd.read_json(json_events)
    df_events_matches = df_events.groupby('matchId', as_index=False)
    for match_id, df_events_match in df_events_matches:
        df_events_match.to_hdf('wyscout.h5', key=f'events/match_{match_id}', mode='a')

## Convert the Wyscout data to the SPADL representation

In [17]:
convert_to_spadl('wyscout.h5', 'spadl.h5')

...Inserting actiontypes
...Inserting bodyparts
...Inserting results
...Converting games
...Converting players


  1%|          | 4/380 [00:00<00:12, 30.52game/s]

...Converting teams
...Generating player_games


  0%|          | 0/380 [00:00<?, ?game/s]

...Converting events to actions


100%|██████████| 380/380 [19:17<00:00,  3.05s/game]


# Value game states

In [18]:
df_games = pd.read_hdf('spadl.h5', key='games')
df_actiontypes = pd.read_hdf('spadl.h5', key='actiontypes')
df_bodyparts = pd.read_hdf('spadl.h5', key='bodyparts')
df_results = pd.read_hdf('spadl.h5', key='results')

In [19]:
nb_prev_actions = 3

## Generate game state features

In [20]:
functions_features = [
    features.actiontype_onehot,
    features.bodypart_onehot,
    features.result_onehot,
    features.goalscore,
    features.startlocation,
    features.endlocation,
    features.movement,
    features.space_delta,
    features.startpolar,
    features.endpolar,
    features.team,
    features.time_delta
]

In [21]:
df_actions = pd.read_hdf('spadl.h5', key=f'actions/game_{2500089}')

In [22]:
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_actions = pd.read_hdf('spadl.h5', key=f'actions/game_{game_id}')
    df_actions = (df_actions
        .merge(df_actiontypes, how='left')
        .merge(df_results, how='left')
        .merge(df_bodyparts, how='left')
        .reset_index(drop=True)
    )
    
    dfs_gamestates = features.gamestates(df_actions, nb_prev_actions=nb_prev_actions)
    dfs_gamestates = features.play_left_to_right(dfs_gamestates, game['home_team_id'])
    
    df_features = pd.concat([function(dfs_gamestates) for function in functions_features], axis=1)
    df_features.to_hdf('features.h5', key=f'game_{game_id}')

## Generate game state labels

In [23]:
functions_labels = [
    labels.scores,
    labels.concedes
]

In [24]:
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_actions = pd.read_hdf('spadl.h5', key=f'actions/game_{game_id}')
    df_actions = (df_actions
        .merge(df_actiontypes, how='left')
        .merge(df_results, how='left')
        .merge(df_bodyparts, how='left')
        .reset_index(drop=True)
    )
    
    df_labels = pd.concat([function(df_actions) for function in functions_labels], axis=1)
    df_labels.to_hdf('labels.h5', key=f'game_{game_id}')

## Generate dataset

In [25]:
columns_features = features.feature_column_names(functions_features, nb_prev_actions=nb_prev_actions)

In [26]:
dfs_features = []
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_features = pd.read_hdf('features.h5', key=f'game_{game_id}')
    dfs_features.append(df_features[columns_features])
df_features = pd.concat(dfs_features).reset_index(drop=True)

In [27]:
df_features.head(10)

,type_pass_a0,type_cross_a0,type_throw_in_a0,type_freekick_crossed_a0,type_freekick_short_a0,type_corner_crossed_a0,type_corner_short_a0,type_take_on_a0,type_foul_a0,type_tackle_a0,type_interception_a0,type_shot_a0,type_shot_penalty_a0,type_shot_freekick_a0,type_keeper_save_a0,type_keeper_claim_a0,type_keeper_punch_a0,type_keeper_pick_up_a0,type_clearance_a0,type_bad_touch_a0,type_non_action_a0,type_dribble_a0,type_goalkick_a0,type_pass_a1,type_cross_a1,type_throw_in_a1,type_freekick_crossed_a1,type_freekick_short_a1,type_corner_crossed_a1,type_corner_short_a1,type_take_on_a1,type_foul_a1,type_tackle_a1,type_interception_a1,type_shot_a1,type_shot_penalty_a1,type_shot_freekick_a1,type_keeper_save_a1,type_keeper_claim_a1,type_keeper_punch_a1,type_keeper_pick_up_a1,type_clearance_a1,type_bad_touch_a1,type_non_action_a1,type_dribble_a1,type_goalkick_a1,type_pass_a2,type_cross_a2,type_throw_in_a2,type_freekick_crossed_a2,type_freekick_short_a2,type_corner_crossed_a2,type_corner_short_a2,type_take_on_a2,type_foul_a2,type_tackle_a2,type_interception_a2,type_shot_a2,type_shot_penalty_a2,type_shot_freekick_a2,type_keeper_save_a2,type_keeper_claim_a2,type_keeper_punch_a2,type_keeper_pick_up_a2,type_clearance_a2,type_bad_touch_a2,type_non_action_a2,type_dribble_a2,type_goalkick_a2,bodypart_foot_a0,bodypart_head_a0,bodypart_other_a0,bodypart_foot_a1,bodypart_head_a1,bodypart_other_a1,bodypart_foot_a2,bodypart_head_a2,bodypart_other_a2,result_fail_a0,result_success_a0,result_offside_a0,result_owngoal_a0,result_yellow_card_a0,result_red_card_a0,result_fail_a1,result_success_a1,result_offside_a1,result_owngoal_a1,result_yellow_card_a1,result_red_card_a1,result_fail_a2,result_success_a2,result_offside_a2,result_owngoal_a2,result_yellow_card_a2,result_red_card_a2,goalscore_team,goalscore_opponent,goalscore_diff,start_x_a0,start_y_a0,start_x_a1,start_y_a1,start_x_a2,start_y_a2,end_x_a0,end_y_a0,end_x_a1,end_y_a1,end_x_a2,end_y_a2,dx_a0,dy_a0,movement_a0,dx_a1,dy_a1,movement_a1,dx_a2,dy_a2,movement_a2,dx_a01,dy_a01,mov_a01,dx_a02,dy_a02,mov_a02,start_dist_to_goal_a0,start_angle_to_goal_a0,start_dist_to_goal_a1,start_angle_to_goal_a1,start_dist_to_goal_a2,start_angle_to_goal_a2,end_dist_to_goal_a0,end_angle_to_goal_a0,end_dist_to_goal_a1,end_angle_to_goal_a1,end_dist_to_goal_a2,end_angle_to_goal_a2,team_1,team_2,time_delta_1,time_delta_2
0,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,True,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,0,0,0,52.50,34.00,52.50,34.00,52.50,34.00,42.00,37.40,42.00,37.40,42.00,37.40,-10.50,3.40,11.036757,-10.50,3.40,11.036757,-10.50,3.40,11.036757,-1.050000e+01,3.400000e+00,1.103676e+01,-10.50,3.400000e+00,11.036757,52.500000,0.000000,52.500000,0.000000,52.500000,0.000000,63.091679,0.053916,63.091679,0.053916,63.091679,0.053916,True,True,0.000000,0.000000
1,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,True,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,0,0,0,42.00,37.40,52.50,34.00,52.50,34.00,40.95,57.80,42.00,37.40,42.00,37.40,-1.05,20.40,20.427004,-10.50,3.40,11.036757,-10.50,3.40,11.036757,0.

In [28]:
columns_labels = [
    'scores',
    'concedes'
]

In [29]:
dfs_labels = []
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_labels = pd.read_hdf('labels.h5', key=f'game_{game_id}')
    dfs_labels.append(df_labels[columns_labels])
df_labels = pd.concat(dfs_labels).reset_index(drop=True)

In [30]:
df_labels.head(10)

,scores,concedes
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
5,False,False
6,False,False
7,False,False
8,False,False
9,False,False


## Train classifiers

In [31]:
models = {}
for column_labels in columns_labels:
    model = XGBClassifier(n_estimators=100, max_depth=4)
    model.fit(df_features, df_labels[column_labels])
    models[column_labels] = model

## Estimate probabilities

In [32]:
dfs_predictions = {}
for column_labels in columns_labels:
    model = models[column_labels]
    probabilities = model.predict_proba(df_features)
    predictions = probabilities[:, 1]
    dfs_predictions[column_labels] = pd.Series(predictions)
df_predictions = pd.concat(dfs_predictions, axis=1)

In [33]:
df_predictions.head(10)

,scores,concedes
0,0.002290,0.000309
1,0.005068,0.002664
2,0.005252,0.001473
3,0.012293,0.001732
4,0.019258,0.000786
5,0.016444,0.001715
6,0.009395,0.016821
7,0.003122,0.020879
8,0.005181,0.001756
9,0.000950,0.013203


The following cell obtains the `game_id` for each action in order to store the predictions per game.

In [34]:
dfs_game_ids = []
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_actions = pd.read_hdf('spadl.h5', key=f'actions/game_{game_id}')
    dfs_game_ids.append(df_actions['game_id'])
df_game_ids = pd.concat(dfs_game_ids, axis=0).astype('int').reset_index(drop=True)

In [35]:
df_predictions = pd.concat([df_predictions, df_game_ids], axis=1)

In [36]:
df_predictions.head(10)

,scores,concedes,game_id
0,0.002290,0.000309,2500089
1,0.005068,0.002664,2500089
2,0.005252,0.001473,2500089
3,0.012293,0.001732,2500089
4,0.019258,0.000786,2500089
5,0.016444,0.001715,2500089
6,0.009395,0.016821,2500089
7,0.003122,0.020879,2500089
8,0.005181,0.001756,2500089
9,0.000950,0.013203,2500089


In [37]:
df_predictions_per_game = df_predictions.groupby('game_id')

In [38]:
for game_id, df_predictions in tqdm(df_predictions_per_game):
    df_predictions = df_predictions.reset_index(drop=True)
    df_predictions[columns_labels].to_hdf('predictions.h5', key=f'game_{game_id}')

# Value on-the-ball actions - VAEP Values

<img src="./expl_photo/1.JPG">
<img src="./expl_photo/2.JPG">
<img src="./expl_photo/3.JPG">
<img src="./expl_photo/4.JPG">

In [39]:
df_players = pd.read_hdf('spadl.h5', key='players')
df_teams = pd.read_hdf('spadl.h5', key='teams')

In [40]:
dfs_values = []
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_actions = pd.read_hdf('spadl.h5', key=f'actions/game_{game_id}')
    df_actions = (df_actions
        .merge(df_actiontypes, how='left')
        .merge(df_results, how='left')
        .merge(df_bodyparts, how='left')
        .merge(df_players, how='left')
        .merge(df_teams, how='left')
        .reset_index(drop=True)
    )
    
    df_predictions = pd.read_hdf('predictions.h5', key=f'game_{game_id}')
    df_values = value(df_actions, df_predictions['scores'], df_predictions['concedes'])
    
    df_all = pd.concat([df_actions, df_predictions, df_values], axis=1)
    dfs_values.append(df_all)

In [41]:
df_values = (pd.concat(dfs_values)
    .sort_values(['game_id', 'period_id', 'time_seconds'])
    .reset_index(drop=True)
)

In [42]:
df_values.loc[(df_values['game_id'] == 2500093) & (df_values['last_name'].str.contains('Kaba'))]

,game_id,period_id,time_seconds,team_id,player_id,start_x,start_y,end_x,end_y,bodypart_id,type_id,result_id,type_name,result_name,bodypart_name,short_name,first_name,last_name,birth_date,short_team_name,team_name,scores,concedes,offensive_value,defensive_value,vaep_value
476456,2500093.0,1.0,606.611590,1644.0,68085.0,79.80,29.24,56.70,51.68,0,0,1,pass,success,foot,C. Kabasele,Christian,Kabasele,1991-02-24,Watford,Watford FC,0.005109,0.004210,0.004299,0.009863,0.014162
476478,2500093.0,1.0,659.001279,1644.0,68085.0,95.55,31.28,105.00,0.00,0,18,0,clearance,fail,foot,C. Kabasele,Christian,Kabasele,1991-02-24,Watford,Watford FC,0.005640,0.019965,0.004269,-0.001934,0.002334
476493,2500093.0,1.0,782.335595,1644.0,68085.0,66.15,17.00,37.80,68.00,1,0,0,pass,fail,head,C. Kabasele,Christian,Kabasele,1991-02-24,Watford,Watford FC,0.002725,0.001847,0.000594,0.002595,0.003189
476541,2500093.0,1.0,944.376720,1644.0,68085.0,79.80,10.20,71.40,17.68,0,0,1,pass,success,foot,C. Kabasele,Christian,Kabasele,1991-02-24,Watford,Watford FC,0.005175,0.001818,0.001837,0.004389,0.006225
476543,2500093.0,1.0,952.212074,1644.0,68085.0,74.55,14.96,46.20,19.72,0,0,1,pass,success,foot,C. Kabasele,Christian,Kabasele,1991-02-24,Watford,Watford FC,0.009281,0.002554,0.004836,-0.001261,0.003574
476547,2500093.0,1.0,960.071365,1644.0,68085.0,75.60,21.08,72.45,46.92,0,0,1,pass,success,foot,C. Kabasele,Christian,Kabasele,1991-02-24,Watford,Watford FC,0.005540,0.002264,0.000268,0.000724,0.000992
476549,2500093.0,1.0,968.364921,1644.0,68085.0,69.30,16.32,45.15,19.72,0,0,0,pass,fail,foot,C. Kabasele,Christian,Kabasele,1991-02-24,Watford,Watford FC,0.003349,0.002784,-0.005535,-0.001588,-0.007123


* scores = P_scores(Si, x) - prob(scoring in the next 10 actions) - obtained from the ML Model - given for the particular `state`
* concedes = P_concedes(Si, x) - prob(concedes in the next 10 actions) - obtained from the ML Model - given for the particular `state`
* offensive_value = P_scores(Si) - P_scores(Si-1) - given for the particular `action`
* defensive_value = P_concedes(Si-1) - P_concedes(Si) - given for the particular `action`

In [43]:
df_values.to_pickle('./vaep_values.pkl')

# VAEP for Defenders 

## Pre-Processing 

In [44]:
vaep_values = pd.read_pickle('./vaep_values.pkl')
match_def = pd.read_pickle("../../data/matches/match+def_lineup+footedness_ver2.pkl")

In [45]:
vaep_values = vaep_values.replace({'short_team_name':{
        'Manchester United': 'Man Utd',
        'Tottenham Hotspur': 'Spurs',
        'West Ham United': 'West Ham',
        'Manchester City': 'Man City',
        'Brighton & Hove Albion': 'Brighton',
        'Stoke City': 'Stoke',
        'AFC Bournemouth': 'Bournemouth',
        'West Bromwich Albion': 'West Brom',
        'Leicester City': 'Leicester',
        'Swansea City': 'Swansea',
        'Huddersfield Town': 'Huddersfield',
        'Newcastle United': 'Newcastle'
    }}
)

vaep_values['game_id'] = vaep_values['game_id'].astype(int)
vaep_values['temp'] = vaep_values['game_id'].astype(str) + vaep_values['short_team_name']
match_def['temp'] = match_def['wyId'].astype(str) + match_def['team']

In [46]:
vaep_values = vaep_values.merge(match_def[['temp', 'footedness']], left_on='temp', right_on='temp', how='left')
vaep_values.drop(columns = ['temp'], inplace=True)

In [47]:
vaep_values['name'] = vaep_values['first_name']+vaep_values['last_name']
vaep_values['name'] = vaep_values['name'].astype(str).apply(lambda x: unidecode(x))
vaep_values['name'] = vaep_values['name'].apply(lambda x: x.replace('-', ''))
vaep_values['name'] = vaep_values['name'].apply(lambda x: x.replace(' ', ''))
vaep_values.loc[(vaep_values['game_id'] == 2500089) & (vaep_values['last_name'].str.contains('Ak'))]

,game_id,period_id,time_seconds,team_id,player_id,start_x,start_y,end_x,end_y,bodypart_id,type_id,result_id,type_name,result_name,bodypart_name,short_name,first_name,last_name,birth_date,short_team_name,team_name,scores,concedes,offensive_value,defensive_value,vaep_value,footedness,name
471161,2500089,1.0,62.385612,1659.0,134102.0,55.65,31.28,65.10,13.60,0,0,1,pass,success,foot,N. Aké,Nathan,Aké,1995-02-18,Bournemouth,AFC Bournemouth,0.007569,0.001214,-0.003466,0.000499,-0.002967,right-left-left,NathanAke
471171,2500089,1.0,102.512177,1659.0,134102.0,98.70,64.60,57.75,61.88,0,0,1,pass,success,foot,N. Aké,Nathan,Aké,1995-02-18,Bournemouth,AFC Bournemouth,0.002449,0.001659,0.001180,0.007529,0.008709,right-left-left,NathanAke
471175,2500089,1.0,158.158523,1659.0,134102.0,76.65,39.44,77.70,63.24,0,0,1,pass,success,foot,N. Aké,Nathan,Aké,1995-02-18,Bournemouth,AFC Bournemouth,0.002881,0.001392,0.000051,0.001535,0.001587,right-left-left,NathanAke
471209,2500089,1.0,303.599394,1659.0,134102.0,74.55,56.44,66.15,68.00,0,18,0,clearance,fail,foot,N. Aké,Nathan,Aké,1995-02-18,Bournemouth,AFC Bournemouth,0.000786,0.006222,-0.001538,-0.003781,-0.005319,right-left-left,NathanAke
471292,2500089,1.0,657.031182,1659.0,134102.0,75.60,29.24,55.65,13.60,0,0,1,pass,success,foot,N. Aké,Nathan,Aké,1995-02-18,Bournemouth,AFC Bournemouth,0.008472,0.001616,0.002838,0.000274,0.003112,right-left-left,NathanAke
471366,2500089,1.0,965.661926,1659.0,134102.0,95.55,51.68,105.00,49.64,0,10,1,interception,success,foot,N. Aké,Nathan,Aké,1995-02-18,Bournemouth,AFC Bournemouth,0.005487,0.020208,0.004129,0.006889,0.011018,right-left-left,NathanAke
471375,2500089,1.0,1068.435383,1659.0,134102.0,76.65,45.56,76.65,45.56,0,10,0,interception,fail,foot,N. Aké,Nathan,Aké,1995-02-18,Bournemouth,AFC Bournemouth,0.002918,0.005305,-0.000183,-0.001911,-0.002093,right-left-left,NathanAke
471376,2500089,1.0,1068.435383,1659.0,134102.0,76.65,45.56,49.35,45.56,1,0,0,pass,fail,head,N. Aké,Nathan,Aké,1995-02-18,Bournemouth,AFC Bournemouth,0.002378,0.007170,-0.000539,-0.001865,-0.002404,right-left-left,NathanAke
471379,2500089,1.0,1074.918617,1659.0,134102.0,72.45,23.80,77.70,19.04,0,0,1,pass,success,foot,N. Aké,Nathan,Aké,1995-02-18,Bournemouth,AFC Bournemouth,0.004886,0.002617,0.001381,0.003781,0.005162,right-left-left,NathanAke
471381,2500089,1.0,1078.695902,1659.0,134102.0,78.75,17.00,75.60,13.60,0,0,1,pass,success,foot,N. Aké,Nathan,Aké,1995-02-18,Bournemouth,AFC Bournemouth,0.005288,0.003489,0.000153,-0.000759,-0.000606,right-left-left,NathanAke


In [48]:
# Creating a key to identify each row using game_id, time_seconds and player name
vaep_values['key'] = vaep_values['game_id'].astype(str) + np.round(vaep_values['time_seconds'],6).astype(str) +vaep_values['name'].astype(str)

In [49]:
# Filtering the events that are labelled as pass and cross
vaep_values_pass = vaep_values.loc[(vaep_values['type_name'] == 'pass') | (vaep_values['type_name'] =='cross') ]

In [50]:
# Merging player roles to SPADL events data
df_players = pd.read_pickle('../../data/players/players.pkl')
roles_temp = df_players['role'].values
roles = list()
for i in roles_temp:
    roles.append(i['code3'])
players_roles = list(zip(roles,df_players['wyId'],df_players['playerName']))
df_players_roles = pd.DataFrame(players_roles,columns = ['role','playerId','playerName1'])
vaep_values_pass_proles = vaep_values_pass.merge(df_players_roles, left_on = 'player_id', right_on = 'playerId')
vaep_values_pass_proles.drop(['playerName1'], axis = 1, inplace = True)
vaep_values_pass_def = vaep_values_pass_proles.loc[vaep_values_pass_proles['role']=='DEF']

In [51]:
# Counting duplicate keys in the VAEP filtered dataframe. 
# Note: Some events in the dataframe have same key attributes (game_id, time_seconds and player name)
# When checked at a deeper level, these passes (roughly 30 of them) have the same player performing two 
# different passes from the same starting location to different end locations at the same time stamp.
# We have ignored these 30 passes from our analysis
vaep_values_pass_def['key'].value_counts()

2499949433.742393LaurentKoscielny          2
2499949483.766015ShkodranMustafi           2
24999722087.498654NicolasHernanOtamendi    2
2499949234.956542HectorBellerinMoruno      2
2499949596.49891LaurentKoscielny           2
                                          ..
25000081099.992603SebastianProdl           1
25000802447.000759CalumChambers            1
2499822227.685429JohnStones                1
2499764615.544184AshleyYoung               1
24997672135.016897KyleWalker               1
Name: key, Length: 141940, dtype: int64

In [52]:
# vaep_values_pass_def.loc[vaep_values_pass_def['key'].str.contains('2499949256.961055LaurentKoscielny')]

In [53]:
# Importing Wyscout events data
df_events_wyscout = pd.read_pickle('../../data/events/events_v2.pkl')

In [54]:
# Filtering out passes events which were performed by defenders in Wyscout events data
df_events_wyscout_pass = df_events_wyscout.loc[(df_events_wyscout['eventName']=='Pass') & (df_events_wyscout['role']=='DEF')]

In [55]:
# Creating a key to identify each row using matchid, event_seconds and player name
df_events_wyscout_pass['key'] = df_events_wyscout_pass['matchId'].astype(str) + np.round(df_events_wyscout_pass['eventSec'],6).astype(str) + df_events_wyscout_pass['playerName'].astype(str)

/Users/shreygrover/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [56]:
# df_events_wyscout_pass.loc[df_events_wyscout_pass['key'].str.contains('2499949256.961055LaurentKoscielny')]

In [57]:
# Appending VAEP values along with offensive and defensive values to Wyscout events data from VAEP data
# by joining the two dataframes
df_events_vaep = df_events_wyscout_pass.merge(vaep_values_pass_def[['key','vaep_value','offensive_value','defensive_value']], how = 'left', left_on = 'key', right_on = 'key')

In [58]:
# Total number of passes in Wyscout events data
len(df_events_vaep)

138564

## MetricsCollection & Clustering

In [59]:
footedness_patterns = match_def["footedness"].unique()

In [60]:
match_def.rename(columns={'R-CB':'R_CB',"L-CB":'L_CB'},inplace=True)

**Creating seperate dataframes for each defensive lineup based for preferred foot of each defender**

In [61]:
df_rrrl = match_def.loc[match_def['footedness']=='right-right-right-left']
df_rrll = match_def.loc[match_def['footedness']=='right-right-left-left']
df_rrl = match_def.loc[match_def['footedness']=='right-right-left']
df_rrr = match_def.loc[match_def['footedness']=='right-right-right']
df_rll = match_def.loc[match_def['footedness']=='right-left-left']
df_rrrll = match_def.loc[match_def['footedness']=='right-right-right-left-left']
df_rrlr = match_def.loc[match_def['footedness']=='right-right-left-right']
df_rrrr = match_def.loc[match_def['footedness']=='right-right-right-right']
df_rrrrl = match_def.loc[match_def['footedness']=='right-right-right-right-left']
df_rlr = match_def.loc[match_def['footedness']=='right-left-right']
df_rrrlr = match_def.loc[match_def['footedness']=='right-right-right-left-right']
df_rrlll = match_def.loc[match_def['footedness']=='right-right-left-left-left']
df_rlll = match_def.loc[match_def['footedness']=='right-left-left-left']

**Creating a list of such dataframes**

In [62]:
df_clusters = [df_rrrl,df_rrll,df_rrl,df_rrr,df_rll,df_rrrll,df_rrlr,df_rrrr,df_rrrrl,df_rlr,df_rrrlr,df_rrlll,df_rlll]

**Creating a dictionary of mapping of players with a mismatch in names in events data and Premier League parsed data**

In [63]:
player_map = {  'RamiroFunesMori': 'JoseRamiroFunesMori',
                'KurtZouma': 'KurtHappyZouma',
                'Danilo': 'DaniloLuizdaSilva',
                'CesarAzpilicueta': 'CesarAzpilicuetaTanco',
                'EzequielSchelotto': 'MatiasEzequielSchelotto',
                'GaetanBong': 'GaetanBongSongo',
                'HectorBellerin': 'HectorBellerinMoruno',
                'AhmedHegazi': 'AhmedHegazy',
                'JamaalLascelles': 'JamalLascelles',
                'AngelRangel': 'AngelRangelZaragoza',
                'Zanka': 'MathiasJattahNjieJorgensen',
                'ChrisLwe': 'ChrisLowe',
                'EricBailly': 'EricBertrandBailly',
                'MarcosRojo': 'FaustinoMarcosAlbertoRojo',
                'CdricSoares': 'CedricRicardoAlvesSoares',
                'AngeloOgbonna': 'AngeloObinzeOgbonna',
                'HctorBellern': 'HectorBellerinMoruno',
                'DavinsonSanchez': 'DavinsonSanchezMina',
                'JavierManquillo': 'JavierManquilloGaitan',
                'TommySmith': 'TomSmith',
                'Bruno': 'BrunoSaltorGrau',
                'GatanBong': 'GaetanBongSongo',
                'NicolsOtamendi': 'NicolasHernanOtamendi',
                'CsarAzpilicueta': 'CesarAzpilicuetaTanco',
                'AntonioRdiger': 'AntonioRudiger',
                'JosHolebas': 'JoseHolebas',
                'SamusColeman': 'SeamusColeman',
                'AllanRomoNyom': 'AllanRomeoNyom',
                'NathanAk': 'NathanAke',
                'JosephGomez': 'JoeGomez',
                'AlbertoMoreno':'AlbertoMorenoPerez',
                'LuisAntonioValencia':'LuisAntonioValenciaMosquera',
                'VictorLindelf':'VictorNilssonLindelof',
                'DavinsonSnchez':'DavinsonSanchezMina',
                'NicolasOtamendi':'NicolasHernanOtamendi',
                'NachoMonreal':'IgnacioMonrealEraso',
                'FedericoFernndez':'FedericoFernandez',
                'SebastianPrdl':'SebastianProdl',
                'CedricSoares':'CedricRicardoAlvesSoares',
                'JoelMatip':'JoelAndreJobMatip',
                'MiguelBritos':'MiguelAngelBritosCabrera',
                'VictorLindelof':'VictorNilssonLindelof',
                'JamesCollins':'JamesMichaelCollins',
                'CucoMartina':'RhuendlyMartina',
                'DavidLuiz':'DavidLuizMoreiraMarinho',
                'MollaWagu':'MollaWague',
                'JrmyPied':'JeremyPied',
                'ChancelMbemba':'ChancelMbembaMangulu',
                'PabloZabaleta':'PabloJavierZabaletaGirod',
                'KikoFemenia':'FranciscoFemeniaFar',
                'CheikhouKouyat':'CheikhouKouyate',
                'KikoFemena':'FranciscoFemeniaFar',
                'JoseFonte':'JoseMigueldaRochaFonte',
                'JosFonte':'JoseMigueldaRochaFonte',
                'JesusGamez':'JesusGamezDuarte'}


**Creating a metrics collection function that takes in x (match_id) and y (player name) and returns the standard metrics-**

In [64]:
def getmetrics(x,y):
    try:
        y = player_map[y]
    except:
        pass
    split_y = re.findall('[A-Z][^A-Z]*',y)
    try:
        pass_df = df_events_vaep.loc[(df_events_vaep['playerName'].str.contains(split_y[-1]))&
                                 (df_events_vaep['playerName'].str.contains(split_y[-2]))&
                                 (df_events_vaep['matchId']==int(x))]
    except:
        pass_df = df_events_vaep.loc[(df_events_vaep['playerName'].str.contains(split_y[-1]))&
                                 (df_events_vaep['matchId']==int(x))]
    numpasses = len(pass_df)
    numaccpasses = len(pass_df.loc[pass_df['tags'].apply(lambda a: "Accurate" in a)])
    accpasslocs = pass_df.loc[pass_df['tags'].apply(lambda a: "Accurate" in a)]['positions'].tolist()
    inaccpasslocs = pass_df.loc[pass_df['tags'].apply(lambda a: "Not accurate" in a)]['positions'].tolist()
    acc_vaep_values = pass_df.loc[pass_df['tags'].apply(lambda a: "Accurate" in a)]['vaep_value'].tolist()
    inacc_vaep_values = pass_df.loc[pass_df['tags'].apply(lambda a: "Not accurate" in a)]['vaep_value'].tolist()
    acc_off_values = pass_df.loc[pass_df['tags'].apply(lambda a: "Accurate" in a)]['offensive_value'].tolist()
    inacc_off_values = pass_df.loc[pass_df['tags'].apply(lambda a: "Not accurate" in a)]['offensive_value'].tolist()
    acc_def_values = pass_df.loc[pass_df['tags'].apply(lambda a: "Accurate" in a)]['defensive_value'].tolist()
    inacc_def_values = pass_df.loc[pass_df['tags'].apply(lambda a: "Not accurate" in a)]['defensive_value'].tolist()
    return [numpasses, numaccpasses, accpasslocs, inaccpasslocs, acc_vaep_values,
            inacc_vaep_values, acc_off_values, inacc_off_values, acc_def_values,
            inacc_def_values]


In [65]:
# getmetrics(2500081,"Bruno")

In [66]:
new_cols = ['RB_all',
            'R_CB_all',
            'L_CB_all',
            'LB_all',
            'RCB_all',
            'CB_all',
            'LCB_all',
            'RWB_all',
            'LWB_all']

**Collecting metrics for each defender location for various clusters**

In [67]:
#R_CB - Right center back for 4 defender formation
#RCB - Right center back for 3 or 5 defender formation
#L_CB - Left center back for 4 defender formation
#LCB - Left center back for 3 or 5 defender formation
df_clusters_updated = list()
for df in tqdm(df_clusters):
    df = df.reindex(columns = df.columns.tolist() + new_cols)
    if df.iloc[0]['backline'] == 4.0:     
        df['RB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.RB), axis=1)
        df['R_CB_all'] = df.apply(lambda x: getmetrics(x.wyId,x['R_CB']), axis=1)
        df['L_CB_all'] = df.apply(lambda x: getmetrics(x.wyId,x['L_CB']), axis=1)
        df['LB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.LB), axis=1)
        df_clusters_updated.append(df)
    
    elif df.iloc[0]['backline'] == 3.0:
        df['RCB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.RCB), axis=1)
        df['CB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.CB), axis=1)
        df['LCB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.LCB), axis=1)
        df_clusters_updated.append(df)
        
    elif df.iloc[0]['backline'] == 5.0:
        df['RWB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.RWB), axis=1)
        df['RCB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.RCB), axis=1)
        df['CB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.CB), axis=1)
        df['LCB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.LCB), axis=1)
        df['LWB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.LWB), axis=1)
        df_clusters_updated.append(df)

**Splitting the metrics into individual columns**

In [68]:
df_clusters_metrics = list()
for df in tqdm(df_clusters_updated):
    if df.iloc[0]['backline'] == 4.0:
        df[[
            'RB_pass', 'RB_accpass', 'RB_accpassloc', 'RB_inaccpassloc',
            'RB_accvaep', 'RB_inaccvaep', 'RB_accoff', 'RB_inaccoff',
            'RB_accdef', 'RB_inaccdef'
        ]] = pd.DataFrame(df['RB_all'].to_list(), index=df.index)
        df[[
            'R_CB_pass', 'R_CB_accpass', 'R_CB_accpassloc',
            'R_CB_inaccpassloc', 'R_CB_accvaep', 'R_CB_inaccvaep',
            'R_CB_accoff', 'R_CB_inaccoff', 'R_CB_accdef', 'R_CB_inaccdef'
        ]] = pd.DataFrame(df['R_CB_all'].to_list(), index=df.index)
        df[[
            'L_CB_pass', 'L_CB_accpass', 'L_CB_accpassloc', 'L_CB_inaccpassloc',
            'L_CB_accvaep', 'L_CB_inaccvaep', 'L_CB_accoff', 'L_CB_inaccoff',
            'L_CB_accdef', 'L_CB_inaccdef'
        ]] = pd.DataFrame(df['L_CB_all'].to_list(), index=df.index)
        df[[
            'LB_pass', 'LB_accpass', 'LB_accpassloc', 'LB_inaccpassloc',
            'LB_accvaep', 'LB_inaccvaep', 'LB_accoff', 'LB_inaccoff',
            'LB_accdef', 'LB_inaccdef'
        ]] = pd.DataFrame(df['LB_all'].to_list(), index=df.index)
        df.drop([
            'RB_all', 'R_CB_all', 'L_CB_all', 'LB_all', 'RCB_all', 'LCB_all',
            'CB_all', 'RWB_all', 'LWB_all'
        ],
                axis=1,
                inplace=True)
        df_clusters_metrics.append(df)

    elif df.iloc[0]['backline'] == 3.0:
        df[[
            'RCB_pass', 'RCB_accpass', 'RCB_accpassloc', 'RCB_inaccpassloc',
            'RCB_accvaep', 'RCB_inaccvaep', 'RCB_accoff', 'RCB_inaccoff',
            'RCB_accdef', 'RCB_inaccdef'
        ]] = pd.DataFrame(df['RCB_all'].to_list(), index=df.index)
        df[[
            'CB_pass', 'CB_accpass', 'CB_accpassloc', 'CB_inaccpassloc',
            'CB_accvaep', 'CB_inaccvaep', 'CB_accoff', 'CB_inaccoff',
            'CB_accdef', 'CB_inaccdef'
        ]] = pd.DataFrame(df['CB_all'].to_list(), index=df.index)
        df[[
            'LCB_pass', 'LCB_accpass', 'LCB_accpassloc', 'LCB_inaccpassloc',
            'LCB_accvaep', 'LCB_inaccvaep', 'LCB_accoff', 'LCB_inaccoff',
            'LCB_accdef', 'LCB_inaccdef'
        ]] = pd.DataFrame(df['LCB_all'].to_list(), index=df.index)
        df.drop([
            'RB_all', 'R_CB_all', 'L_CB_all', 'LB_all', 'RCB_all', 'LCB_all',
            'CB_all', 'RWB_all', 'LWB_all'
        ],
                axis=1,
                inplace=True)
        df_clusters_metrics.append(df)

    elif df.iloc[0]['backline'] == 5.0:
        df[[
            'RCB_pass', 'RCB_accpass', 'RCB_accpassloc', 'RCB_inaccpassloc',
            'RCB_accvaep', 'RCB_inaccvaep', 'RCB_accoff', 'RCB_inaccoff',
            'RCB_accdef', 'RCB_inaccdef'
        ]] = pd.DataFrame(df['RCB_all'].to_list(), index=df.index)
        df[[
            'CB_pass', 'CB_accpass', 'CB_accpassloc', 'CB_inaccpassloc',
            'CB_accvaep', 'CB_inaccvaep', 'CB_accoff', 'CB_inaccoff',
            'CB_accdef', 'CB_inaccdef'
        ]] = pd.DataFrame(df['CB_all'].to_list(), index=df.index)
        df[[
            'LCB_pass', 'LCB_accpass', 'LCB_accpassloc', 'LCB_inaccpassloc',
            'LCB_accvaep', 'LCB_inaccvaep', 'LCB_accoff', 'LCB_inaccoff',
            'LCB_accdef', 'LCB_inaccdef'
        ]] = pd.DataFrame(df['LCB_all'].to_list(), index=df.index)
        df[[
            'RWB_pass', 'RWB_accpass', 'RWB_accpassloc', 'RWB_inaccpassloc',
            'RWB_accvaep', 'RWB_inaccvaep', 'RWB_accoff', 'RWB_inaccoff',
            'RWB_accdef', 'RWB_inaccdef'
        ]] = pd.DataFrame(df['RWB_all'].to_list(), index=df.index)
        df[[
            'LWB_pass', 'LWB_accpass', 'LWB_accpassloc', 'LWB_inaccpassloc',
            'LWB_accvaep', 'LWB_inaccvaep', 'LWB_accoff', 'LWB_inaccoff',
            'LWB_accdef', 'LWB_inaccdef'
        ]] = pd.DataFrame(df['LWB_all'].to_list(), index=df.index)
        df.drop([
            'RB_all', 'R_CB_all', 'L_CB_all', 'LB_all', 'RCB_all', 'LCB_all',
            'CB_all', 'RWB_all', 'LWB_all'
        ],
                axis=1,
                inplace=True)
        df_clusters_metrics.append(df)

In [69]:
# df_clusters_metrics[0].head()

In [70]:
# Saving the clusters into individual pickle files
names = ['rrrl','rrll','rrl','rrr','rll','rrrll','rrlr','rrrr','rrrrl','rlr','rrrlr','rrlll','rlll']

for i,df in enumerate(df_clusters_metrics):
    df.to_pickle(f'../../data/clusters/clusters_vaep/cluster_{names[i]}.pkl')

In [71]:
# rrrl = pd.read_pickle(f'../../data/clusters/clusters_vaep/cluster_rrrl.pkl')
# rrrl.head()